In [ ]:
import numpy as np
from src.evaluation import Evaluator
from tqdm import tqdm

In [ ]:
task = "reach_top_right" # "reach_top_left"
model= "td_cfm" # "td2_cfm"
epoch = None

In [9]:
evaluator = Evaluator(model=model, task=task, epoch=epoch, num_samples=64, disable=True)

td_cfm_model_reach_top_right.pth:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

In [10]:
estimates = [evaluator.evaluate() for _ in tqdm(range(10), desc="Evaluations")]

Evaluations: 100%|██████████| 10/10 [10:41<00:00, 64.18s/it]


In [11]:
emd = [res['EMD'] for res in estimates]
msev = [res['MSE(V)'] for res in estimates]
nll = [res['NLL'] for res in estimates]

In [12]:
print("Task:", task)
print("Model:", model)
print("EMD:", np.mean(emd), f'({np.std(emd)})')
print("MSE(V):", np.mean(msev), f'({np.std(msev)})')
print("NLL:", np.mean(nll), f'({np.std(nll)})')

Task: reach_top_right
Model: td_cfm
EMD: 0.23702374565963852 (0.0017435010247033916)
MSE(V): 390.04064293148537 (27.56939728297719)
NLL: -0.5971424281597137 (0.04851244384770964)
